<h3 style="color:rgb(0,120,170)">RCE Sample:</h3>

The dataset consists of images of musical notes labeled as Eight, Half, Quarter, Sixteenth, and Whole. The code carries out the following steps:

Import necessary libraries: The script imports required libraries such as os, numpy, pandas, matplotlib, PIL (Python Imaging Library), torch, and scikit-learn.

Load the dataset and create labels: The code defines a dictionary for label encoding and decoding

Split the dataset: The dataset is split into training and testing sets using scikit-learn's train_test_split function.

Load and preprocess images: The images are loaded using the PIL library, resized, and normalized to have values between -1 and 1.

Define RCE functions: Two functions, euclidean_distance and rce, are defined to compute the Euclidean distance between two points and calculate the RCE value

Further split the dataset: The preprocessed images and their corresponding labels are split again into training and testing sets.

Set the theta parameter: The theta parameter is set to a value of 0.1. This can be adjusted based on the dataset.

Train the RCE classifier: The RCE values for each image in the training set are computed and stored in a list.

Classify the images in the testing set: The RCE values for each image in the testing set are computed and compared with the stored RCE values from the training set. 



In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from sklearn.model_selection import train_test_split

# Load dataset and create labels
label_code = {'Eight': 0, 'Half': 1, 'Quarter': 2, 'Sixteenth': 3, 'Whole': 4}
label_decode = ['Eight', 'Half', 'Quarter', 'Sixteenth', 'Whole']
df = pd.DataFrame(columns=['path', 'label'])

for dirname, _, filenames in os.walk('filesDir'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        name = dirname.split('\\')[-1]  # Change the separator to '\\'
        label = label_code[name]
        new_row = pd.DataFrame({'path': [path], 'label': [label]})
        df = pd.concat([df, new_row], ignore_index=True)

df.head()

# Split the dataset into training and testing sets
train, test = train_test_split(df, test_size=0.2, random_state=77)

# Load and preprocess images
image_size = (100, 100)
images = []
labels = []

for index, row in df.iterrows():
    img = Image.open(row['path']).convert('RGB').resize(image_size)
    img = np.array(img, dtype='float32')
    img = 1 - img / 127.5
    images.append(img)
    labels.append(row['label'])

images = np.array(images)

# RCE functions
def euclidean_distance(a, b):
    return torch.sqrt(torch.sum((a - b) ** 2))

def rce(a, b, theta):
    distance = euclidean_distance(a, b)
    return (1 / distance) * torch.exp(-distance / theta)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Move data to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
X_train = torch.tensor(X_train, device=device)
X_test = torch.tensor(X_test, device=device)

theta = 0.1  # You can adjust this parameter based on your dataset
RCE_values = []

# Train the RCE classifier
for i in range(len(X_train)):
    rce_val = rce(X_train[i], X_train, theta)
    RCE_values.append(rce_val)

RCE_values = torch.stack(RCE_values)

# Classify the images in the testing set
predictions = []

for i in range(len(X_test)):
    rce_val = rce(X_test[i], X_train, theta)
    closest_class = torch.argmin(torch.abs(RCE_values - rce_val))
    predictions.append(y_train[closest_class])

# Calculate the classification accuracy
accuracy = np.sum(np.array(predictions) == np.array(y_test)) / len(y_test)
print("Classification accuracy:", accuracy)


cpu
Classification accuracy: 0.192


In [27]:
from PIL import Image

# Load and preprocess new unlabeled images
unlabeled_images_path = 'filesDir'
unlabeled_image_size = (100, 100)
unlabeled_images = []

for filename in os.listdir(unlabeled_images_path):
    if filename.endswith('.bmp'):
        path = os.path.join(unlabeled_images_path, filename)
        img = Image.open(path).convert('RGB').resize(unlabeled_image_size)
        img = np.array(img, dtype='float32')
        img = 1 - img / 127.5
        unlabeled_images.append(img)

unlabeled_images = np.array(unlabeled_images)


# Move unlabeled images to GPU if available
unlabeled_images = torch.tensor(unlabeled_images, device=device)

# Classify the unlabeled images
unlabeled_predictions = []

for i in range(len(unlabeled_images)):
    rce_val = rce(unlabeled_images[i], X_train, theta)
    closest_class = torch.argmin(torch.abs(RCE_values - rce_val))
    unlabeled_predictions.append(y_train[closest_class])

# Print the predicted labels for the new images
for i, prediction in enumerate(unlabeled_predictions):
    print(f"Image {i+1}: {label_decode[prediction]}")

# If you have ground truth labels for the new images, you can calculate the classification accuracy
# Replace `ground_truth_labels` with the actual labels for the new images
ground_truth_labels = []  # Replace this with the actual labels for the new images

if ground_truth_labels:
    accuracy = np.sum(np.array(unlabeled_predictions) == np.array(ground_truth_labels)) / len(ground_truth_labels)
    print("Classification accuracy for unlabeled images:", accuracy)


Image 1: Whole
Image 2: Whole
Image 3: Whole
Image 4: Whole
Image 5: Whole
Image 6: Whole
Image 7: Whole
Image 8: Whole
Image 9: Whole
Image 10: Whole
Image 11: Whole
Image 12: Whole
Image 13: Whole
Image 14: Whole
Image 15: Whole
Image 16: Whole
Image 17: Whole
Image 18: Whole
Image 19: Whole
Image 20: Whole
Image 21: Whole
Image 22: Whole
Image 23: Whole
Image 24: Whole
Image 25: Whole
Image 26: Whole
Image 27: Whole
Image 28: Whole
Image 29: Whole
Image 30: Whole
Image 31: Whole
Image 32: Whole
Image 33: Whole
Image 34: Whole
Image 35: Whole
Image 36: Whole
Image 37: Whole
Image 38: Whole
Image 39: Whole
Image 40: Whole
Image 41: Whole
Image 42: Whole
Image 43: Whole
Image 44: Whole
Image 45: Whole
Image 46: Whole
Image 47: Whole
Image 48: Whole
Image 49: Whole
Image 50: Whole
Image 51: Whole
Image 52: Whole
Image 53: Whole
Image 54: Whole
Image 55: Whole
Image 56: Whole
Image 57: Whole
Image 58: Whole
Image 59: Whole
Image 60: Whole
Image 61: Whole
Image 62: Whole
Image 63: Whole
I